In [1]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

c:\Users\achauha2\OneDrive - Insight\Projects\rag_evaluation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [2]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["azure_endpoint"] = "https://ai-dev-openai-fts.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "7439c61b0745446dbbb029521f498265"

In [3]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
]

c:\Users\achauha2\OneDrive - Insight\Projects\rag_evaluation\venv\Lib\site-packages\langchain_community\embeddings\openai.py:336: UserWarning: If you have openai>=1.0.0 installed and are using Azure, please use the `AzureOpenAIEmbeddings` class.
  warnings.warn(


In [4]:
# from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI
# from langchain.embeddings import AzureOpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
from ragas.llms import LangchainLLM

# Import evaluate before patching the RagasLLM instance
from ragas import evaluate

In [5]:
azure_model = AzureChatOpenAI(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
    azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
    openai_api_version = "2023-07-01-preview",
    openai_organization = "azure",
    temperature=0,    
    deployment_name="gpt35")

In [6]:
# wrapper around azure_model
ragas_azure_model = LangchainLLM(azure_model)
# patch the new RagasLLM instance
answer_relevancy.llm = ragas_azure_model

In [7]:
# init and change the embeddings
# only for answer_relevancy
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
            azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
            openai_api_type = "azure",
            openai_api_version = "2023-07-01-preview",
            deployment="text-embedding-ada-002", 
            chunk_size=1
                                        )
# embeddings can be used as it is
answer_relevancy.embeddings = azure_embeddings

In [8]:
for m in metrics:
    m.__setattr__("llm", ragas_azure_model)

In [9]:
result = evaluate(
    fiqa_eval["baseline"],
    metrics=metrics,
)

result

evaluating with [faithfulness]


100%|██████████| 2/2 [00:18<00:00,  9.37s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [00:37<00:00, 18.89s/it]


evaluating with [context_recall]


...

evaluating with [context_precision]


100%|██████████| 2/2 [00:03<00:00,  1.97s/it]


evaluating with [harmfulness]


100%|██████████| 2/2 [00:03<00:00,  1.83s/it]


{'faithfulness': 0.7107, 'answer_relevancy': 0.8545, 'context_recall': 0.6903, 'context_precision': 0.0667, 'harmfulness': 0.0000}

In [10]:
df = result.to_pandas()
df.head()

question  \
0  How to deposit a cheque issued to an associate...   
1  Can I send a money order from USPS as a business?   
2  1 EIN doing business under multiple business n...   
3         Applying for and receiving business credit   
4               401k Transfer After Business Closure   

                                       ground_truths  \
0  [Have the check reissued to the proper payee.J...   
1  [Sure you can.  You can fill in whatever you w...   
2  [You're confusing a lot of things here. Compan...   
3  ["I'm afraid the great myth of limited liabili...   
4  [You should probably consult an attorney. Howe...   

                                              answer  \
0  \nThe best way to deposit a cheque issued to a...   
1  \nYes, you can send a money order from USPS as...   
2  \nYes, it is possible to have one EIN doing bu...   
3  \nApplying for and receiving business credit c...   
4  \nIf your employer has closed and you need to ...   

                                            contexts  faithfulness  \
0  [Just have the associate sign the back and the...      0.666667   
1  [Sure you can.  You can fill in whatever you w...      0.800000   
2  [You're confusing a lot of things here. Compan...      0.500000   
3  [Set up a meeting with the bank that handles y...      1.000000   
4  [The time horizon for your 401K/IRA is essenti...      0.750000   

   answer_relevancy  context_recall  context_precision  harmfulness  
0          0.982503        0.857143                0.0            0  
1          0.973138        1.000000                0.0            0  
2          0.948871        1.000000                0.0            0  
3          0.912932        1.000000                0.0            0  
4          0.894759        0.000000                0.0            0

In [11]:
result 

{'faithfulness': 0.7107, 'answer_relevancy': 0.8545, 'context_recall': 0.6903, 'context_precision': 0.0667, 'harmfulness': 0.0000}